In [ ]:
!pip install gradio torch transformers

In [3]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Define the model and tokenizer
model_name = "ibm-granite/granite-3.2-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def generate_response(prompt, max_length=512):
    """Generates a text response from the model."""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)

    if torch.cuda.is_available():
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.replace(prompt, "").strip()
    return response

def concept_explanation(concept):
    """Explains a concept in detail."""
    prompt = f"explain the concept of {concept} in detail with example"
    return generate_response(prompt, max_length=800)

def quiz_generator(concept):
    """Generates a quiz for a given concept."""
    prompt = f"generate 5 quiz questions about {concept} with different question types (multiple choice, true/false, short answer). Give me the answers at the end."
    return generate_response(prompt, max_length=1200)

# Create the Gradio interface
with gr.Blocks() as app:
    gr.Markdown("# Educational AI Assistant")
    with gr.Tabs():
        with gr.Tab(label="Concept Explanation"):
            concept_input = gr.Textbox(label="Enter a concept", placeholder="e.g., Machine Learning")
            explain_btn = gr.Button("Explain")
            explanation_output = gr.Textbox(label="Explanation", lines=10)

            # Use the corrected variable name `concept_input`
            explain_btn.click(concept_explanation, inputs=concept_input, outputs=explanation_output)

        with gr.Tab(label="Quiz Generator"):
            quiz_input = gr.Textbox(label="Enter a concept", placeholder="e.g., Machine Learning")
            quiz_btn = gr.Button("Generate Quiz")
            quiz_output = gr.Textbox(label="Quiz Questions & Answers", lines=15)

            # Use the corrected variable name `quiz_input`
            quiz_btn.click(quiz_generator, inputs=quiz_input, outputs=quiz_output)

app.launch(share=True)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://74539b6c1ce5ed9dd5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
